In [41]:
# -*- coding: utf-8 -*- 
# @Time : 2024/2/5 16:00
# @Author : Ismail Bibers
# This code is written in Python 3.7. To install the required packages, run the following commands:
# pip install pandas numpy matplotlib seaborn scikit-learn sympy
# This code is applicable to the Simargl 2022 dataset. 
# implemented Advanced Ensemble techniques: Blending


import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Load the dataset
df = pd.read_csv('/home/ibibers@ads.iu.edu/IDS_Datasets/Combined_datasets/Simargl2022_combined_dataset.csv')

# Drop duplicates: same observation but multiple occurences
main_df_1 = df.drop_duplicates(keep='first')

# Drop columns that have just one unique value, the model wont learn form columns like this.
one_value = main_df_1.columns[main_df_1.nunique() == 1]
main_df_2 = main_df_1.drop(columns = one_value, axis=1)

In [ ]:
# Take random samples of the dataset
# sample_df_1 = main_df_2.sample(frac=0.6, random_state=42)
sample_df_1 = main_df_2.copy()

In [ ]:
# Extract subsample of data
print (" Extract subsample of data: ")
print(sample_df_1['ALERT'].value_counts())

In [ ]:
# Git the features
# print (main_df_2.head())

In [ ]:
# sample_df_1.info()

In [ ]:
# # To see the features that are need to be encoded "Which are not numerical"
# categorical_columns = sample_df_1.select_dtypes(include=['object']).columns
# print("Categorical Columns:", categorical_columns)

In [ ]:
# # Count the numner of the novsalues in the categorical columns
# print(sample_df_1.isna().sum())

In [ ]:
# # Print the columns with leading spaces, if any
# columns_with_leading_spaces = [col for col in main_df_2.columns if col.startswith(' ')]

# if columns_with_leading_spaces:
#     print("Columns with leading spaces:", columns_with_leading_spaces)
# else:
#     print("No columns with leading spaces found.")

# # Remove leading space character in all feature names
# # main_df_2.rename(columns=lambda x: x.lstrip(), inplace=True)


In [ ]:
# sample_df_1 = sample_df_1.drop(['ANOMALY', 'IPV4_SRC_ADDR'] , axis=1)
median_anomaly = sample_df_1['ANOMALY'].median()
sample_df_1['ANOMALY'].fillna(median_anomaly, inplace=True)

In [ ]:
sample_df_1['ALERT'].fillna('Normal', inplace=True)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

categorical_columns = ['PROTOCOL_MAP', 'IPV4_DST_ADDR', 'IPV4_SRC_ADDR']

# Create a copy of the original DataFrame to avoid modifying the original data
df_encoded = sample_df_1.copy()

# Initialize the OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Fit and transform the categorical columns
df_encoded[categorical_columns] = ordinal_encoder.fit_transform(sample_df_1[categorical_columns])

In [ ]:
# # Fill nan values
# print(sample_df_1.isna().sum())

In [ ]:
print("Sampled Dataset Overview:")

dropped_df = ['ALERT']
X = df_encoded.drop(dropped_df, axis=1)
y = df_encoded['ALERT']

In [ ]:
# Initial train test split set and split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# the base models
rf_model = RandomForestClassifier(random_state=42)
mlpc_model = MLPClassifier(random_state=42)
Lr_model = LogisticRegression(random_state=42)
dst_model = DecisionTreeClassifier(random_state=42)

# Print the number of samples before training
print("Number of samples before training:", len(X_train))

In [ ]:
# Train base models on the training set
rf_model.fit(X_train, y_train)
mlpc_model.fit(X_train, y_train)
Lr_model.fit(X_train, y_train)
dst_model.fit(X_train, y_train)

In [ ]:
# predictions on the test set
pred1 = rf_model.predict(X_test)
pred2 = mlpc_model.predict(X_test)
pred3 = Lr_model.predict(X_test)
pred4 = dst_model.predict(X_test)

In [ ]:
# here is the blending part starts, create a new dataset with the predictions of base models
blend_X_train = np.column_stack((pred1, pred2, pred3, pred4))

# Define a meta-model for blending
# meta_model = LogisticRegression(random_state=42)
meta_model = DecisionTreeClassifier(random_state=42)

# Train the meta-model on the blended predictions
meta_model.fit(blend_X_train, y_test)

# Now, let's make predictions on the test set using the base models
base_rf_model_pred = rf_model.predict(X_test)
base_mlpc_model_pred = mlpc_model.predict(X_test)
base_Lr_model_pred = Lr_model.predict(X_test)
base_dst_model_pred = dst_model.predict(X_test)

# Create a new dataset with the predictions of base models for the meta-model
blend_X_test = np.column_stack(
    (base_rf_model_pred, base_mlpc_model_pred, base_Lr_model_pred, base_dst_model_pred))

# predictions on the test set using the meta-model
y_pred_encoded = meta_model.predict(blend_X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_encoded)
print(f"Accuracy: {accuracy}")

In [ ]:
#  Classificaiton Report 
print("\nClassification Report:")
print(classification_report(y_test, y_pred_encoded))


In [ ]:
import numpy as np
# Confusion matrix with string labels
conf_mat = confusion_matrix(y_test, y_pred_encoded)

# Get unique class labels from y_test and y_pred_encoded
unique_labels = np.unique(np.concatenate((y_test, y_pred_encoded)))

# Plotting the Confusion Matrix with class labels
plt.figure(figsize=(6, 6))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=unique_labels, yticklabels=unique_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:

# Extract feature importances from base models
base_models_importances = {}

for i, model in enumerate([rf_model, Lr_model, dst_model, mlpc_model]):
    if hasattr(model, 'feature_importances_'):
        base_models_importances[f'base_model_{i}'] = model.feature_importances_

# Create a DataFrame for visualization
importance_df = pd.DataFrame(base_models_importances, index=X.columns)

# Calculate average feature importance across base models
average_importance = importance_df.mean(axis=1)

# Sort features by average importance
sorted_importance = average_importance.sort_values(ascending=False)

# Visualize the feature importances
plt.figure(figsize=(14, 20))
sns.barplot(x=sorted_importance.values,
            y=sorted_importance.index, palette='mako')
plt.xlabel('Average Importance Value')
plt.ylabel('Feature Name')
plt.title('Average Feature Importance in Blending Model')
plt.show()